<a href="https://colab.research.google.com/github/rajshah0904/ImageClassiferCNN/blob/main/ImageClassiferCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Loading and Normalizing the CIFAR10 Dataset**


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

torch.device("cuda")

transform = transforms.Compose([transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('bird', 'cat', 'deer', 'dog', 'frog', 'horse',
           'ship', 'plane', 'car', 'truck')


Files already downloaded and verified
Files already downloaded and verified


**Define a Convolutional Neural Network for this Dataset**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    # This function initalizes the neural network
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # This function defines the forward propogation
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

**Choosing Loss Function and Optimizer**



In [ ]:
import torch.optim as optim

#Using a Cross Entropy Loss function since we are trying to classify images into categories
criterion = nn.CrossEntropyLoss()
#Using Stochastic Gradient Descent to train NN
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

**Training the CNN**

In [10]:
num_epochs = 4
# Looping over the dataset multiple times to improve accuracy
for epoch in range(num_epochs):  

    running_loss = 0.0 #total loss variable
    for i, data in enumerate(trainloader, 0):
        # Get inputs and labels from data
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward prop + Backward prop + Optimization
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print overall loss every 1000 batches
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 2000] loss: 1.496
[1, 4000] loss: 1.448
[1, 6000] loss: 1.405
[1, 8000] loss: 1.386
[1, 10000] loss: 1.344
[1, 12000] loss: 1.345
[2, 2000] loss: 1.263
[2, 4000] loss: 1.243
[2, 6000] loss: 1.232
[2, 8000] loss: 1.230
[2, 10000] loss: 1.248
[2, 12000] loss: 1.195
[3, 2000] loss: 1.142
[3, 4000] loss: 1.129
[3, 6000] loss: 1.152
[3, 8000] loss: 1.127
[3, 10000] loss: 1.130
[3, 12000] loss: 1.134
[4, 2000] loss: 1.037
[4, 4000] loss: 1.054
[4, 6000] loss: 1.063
[4, 8000] loss: 1.048
[4, 10000] loss: 1.055
[4, 12000] loss: 1.093
Finished Training


**Test CNN Model**

In [13]:
#This code snippet calculates overall accuracy of model at 60%
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3fc1448280><function _MultiProcessingDataLoaderIter.__del__ at 0x7f3fc1448280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__

Traceback (most recent call last):
      File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
self._shutdown_workers()    
self._shutdown_workers()  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers

      File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    if w.is_alive():
assert self._parent_pid == os.getpid(), 'can only test a child process'  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in 

Accuracy of the network on the 10000 test images: 60 %


In [14]:
#Tests model accuracy for each class in dataset
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# Prints accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3fc1448280>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
self._shutdown_workers()

if w.is_alive():      File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
      File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: Exception ignored in: can only test a child process<function _MultiProcessingDataLoaderIter.__del__ at 0x7f3fc1448280>

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Accuracy for class: bird  is 66.5 %
Accuracy for class: cat   is 78.5 %
Accuracy for class: deer  is 27.4 %
Accuracy for class: dog   is 56.5 %
Accuracy for class: frog  is 70.5 %
Accuracy for class: horse is 38.5 %
Accuracy for class: ship  is 56.5 %
Accuracy for class: plane is 69.8 %
Accuracy for class: car   is 68.2 %
Accuracy for class: truck is 67.9 %


**Save CNN Model**

In [15]:
torch.save(net.state_dict(), './ImageClassiferCNN.pth')